In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
# reproducibility
seed = 42
import numpy as np
np.random.seed(seed)
import torch
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
import torchvision
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
device = torch.device("cuda:0")

# **Preprocessing**

In [0]:
!wget https://people.eecs.berkeley.edu/~taesung_park/CycleGAN/datasets/apple2orange.zip
! unzip apple2orange.zip
!rm -f apple2orange.zip
!mkdir -p apple2orange/train apple2orange/test
!mv apple2orange/trainA apple2orange/train/A
!mv apple2orange/trainB apple2orange/train/B
!mv apple2orange/testA apple2orange/test/A
!mv apple2orange/testB apple2orange/test/B

--2020-01-18 19:32:55--  https://people.eecs.berkeley.edu/~taesung_park/CycleGAN/datasets/apple2orange.zip
Resolving people.eecs.berkeley.edu (people.eecs.berkeley.edu)... 128.32.189.73
Connecting to people.eecs.berkeley.edu (people.eecs.berkeley.edu)|128.32.189.73|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 78456409 (75M) [application/zip]
Saving to: ‘apple2orange.zip’

apple2orange.zip    100%[===================>]  74.82M  18.0MB/s    in 4.4s    

2020-01-18 19:33:00 (16.9 MB/s) - ‘apple2orange.zip’ saved [78456409/78456409]

Archive:  apple2orange.zip
   creating: apple2orange/
   creating: apple2orange/trainA/
  inflating: apple2orange/trainA/n07740461_6908.jpg  
  inflating: apple2orange/trainA/n07740461_7635.jpg  
  inflating: apple2orange/trainA/n07740461_586.jpg  
  inflating: apple2orange/trainA/n07740461_9813.jpg  
  inflating: apple2orange/trainA/n07740461_6835.jpg  
  inflating: apple2orange/trainA/n07740461_2818.jpg  
  inflating: apple2orang

In [0]:
import glob
import random
import os

from torch.utils.data import Dataset
from PIL import Image
import torchvision.transforms as transforms

class ImageDataset(Dataset):
    def __init__(self, data_path, transforms_=[transforms.ToTensor()]):
        self.transform = transforms.Compose(transforms_)
        # siempre tener los fichero ordenados por orden alfabetico, así el shuffle es replicable
        self.files = sorted(glob.glob(data_path + '/*.jpg'))

    def __getitem__(self, index):
        item = self.transform(Image.open(self.files[index]))
        return item

    def __len__(self):
        return len(self.files)

In [0]:
transforms_test = [ 
                   transforms.ToTensor(), 
                   transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5)), 
                  ]

transforms_train = [ 
                   #transforms.Resize(int(256*1.12), Image.BICUBIC), 
                   transforms.RandomCrop(256), 
                   transforms.RandomHorizontalFlip(),
                   transforms.ToTensor(), 
                   transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5)), 
                  ]                

batch_size_ = 16

dataloader_train_apple = DataLoader(ImageDataset("./apple2orange/train/A", transforms_=transforms_test), batch_size=batch_size_, shuffle=False, num_workers=0)
dataloader_train_orange = DataLoader(ImageDataset("./apple2orange/train/B", transforms_=transforms_test), batch_size=batch_size_, shuffle=False, num_workers=0)
dataloader_test_apple = DataLoader(ImageDataset("./apple2orange/test/A", transforms_=transforms_train), batch_size=batch_size_, shuffle=True, num_workers=0)
dataloader_test_orange = DataLoader(ImageDataset("./apple2orange/test/B", transforms_=transforms_train), batch_size=batch_size_, shuffle=True, num_workers=0)

# **Discriminator**

In [0]:
import torch.nn as nn
import torch.nn.functional as F

def conv(in_channels, out_channels, kernel_size, stride=2, padding=1, batch_norm=True):
  layers = [
            nn.Conv2d(in_channels=in_channels, out_channels=out_channels, kernel_size=kernel_size, stride=stride, padding=padding),
          ]
  if batch_norm:
    layers.append(nn.InstanceNorm2d(out_channels))
  return nn.Sequential(*layers)

In [0]:
class Discriminator(nn.Module):
  def __init__(self):
    super().__init__()
    self.block_1 = conv(3, 64, 4, batch_norm=False)
    self.block_2 = conv(64, 128, 4)
    self.block_3 = conv(128, 256, 4)
    self.block_4 = conv(256, 512, 4)
    self.block_5 = conv(512, 1, 4, 1, batch_norm=False)
  
  def forward(self, x):
    # x: [:, 3, 256, 256]
    s1 = F.leaky_relu_(self.block_1(x), negative_slope=0.2) # s1: [:, 64, 128, 128]
    s2 = F.leaky_relu_(self.block_2(s1), negative_slope=0.2) # s2: [:, 168, 64, 64]
    s3 = F.leaky_relu_(self.block_3(s2), negative_slope=0.2) # s3: [:, 256, 32, 32]
    s4 = F.leaky_relu_(self.block_4(s3), negative_slope=0.2) # s4: [:, 512, 16, 16]
    s5 = self.block_5(s4) # s5: [:, 1, 15, 15]
    #return F.avg_pool2d(s5, s5.size()[2:]).view(s5.size()[0], -1) # [:, 1]
    return F.avg_pool2d(s5, s5.size()[2:]).view(s5.size()[0], -1).flatten(start_dim=0) # [:]

# **Generator**

In [0]:
class ResidualBlock(nn.Module):
  def __init__(self):
    super().__init__()
    self.block_1 = conv(256, 256, 3, 1, 1)
    self.block_2 = conv(256, 256, 3, 1, 1)
  
  def forward(self, x):
    # x: [:, 3, 256, 256]
    s1 = F.relu(self.block_1(x)) # s1: [:, 256, 32, 32]
    s2 = x + self.block_2(s1) # s2: [:, 256, 32, 32]
    return s2
    

In [0]:
def deconv(in_channels, out_channels, kernel_size, stride=2, padding=1, output_padding=0, batch_norm=True):
  layers = [
            nn.ConvTranspose2d(in_channels, out_channels, kernel_size, stride, padding, output_padding),
  ]
  if batch_norm:
    layers.append(nn.InstanceNorm2d(out_channels))
  return nn.Sequential(*layers)

In [0]:
class Generator(nn.Module):
  
  def __init__(self):
    super().__init__()
    # Encoder
    self.block_1 = conv(3, 64, 4) # original 7x7 stride 1 (no cabe colab)
    self.block_2 = conv(64, 128, 4)
    self.block_3 = conv(128, 256, 4)
    # Transformer
    rn_blocks_ = []
    for _ in range(6):
      rn_blocks_.append(ResidualBlock())
    self.rn_blocks = nn.Sequential(*rn_blocks_)
    # Decoder
    self.block_4 = deconv(256, 128, 4)
    self.block_5 = deconv(128, 64, 4)
    self.block_6 = deconv(64, 3, 4, batch_norm=False) # original 7x7 stride 1 (no cabe colab)
  
  def forward(self, x):
    # x: [:, 3, 256, 256]
    s1 = F.relu(self.block_1(x)) # s1: [:, 64, 128, 128]
    s2 = F.relu(self.block_2(s1)) # s2: [:, 128, 64, 64]
    s3 = F.relu(self.block_3(s2)) # s3: [:, 256, 32, 32]
    s4 = self.rn_blocks(s3) # s4: [:, 256, 32, 32]
    s5 = F.relu(self.block_4(s4)) # s5: [:, 128, 64, 64]
    s6 = F.relu(self.block_5(s5)) # s6: [:, 64, 128, 128]
    s7 = torch.tanh(self.block_6(s6)) # s7: [:, 3, 256, 256]
    return s7

# **CycleGAN**

In [0]:
model_name="CycleGan_da"
def create_CyCleGAN():
  G_XtoY = Generator().to(device)
  G_YtoX = Generator().to(device)
  D_X = Discriminator().to(device)
  D_Y = Discriminator().to(device)
  return G_XtoY, G_YtoX, D_X, D_Y

G_XtoY, G_YtoX, D_X, D_Y = create_CyCleGAN()

# **Train**

In [0]:
# decrecar learning rate en cada epoch a partir de una época concreta
# cuando ya se tiene algo entrenado
# parecido a multisteplr
class LRLambda():
    def __init__(self, num_training_epochs, decay_start_epoch):
        self.num_training_epochs = num_training_epochs
        self.decay_start_epoch = decay_start_epoch

    def step(self, current_epoch):
        return 1.0 - max(0, current_epoch - self.decay_start_epoch)/(self.num_training_epochs - self.decay_start_epoch)

In [0]:
criterion_GAN = torch.nn.MSELoss()
criterion_cycle = torch.nn.L1Loss()
criterion_identity = torch.nn.L1Loss()

In [0]:
import torch.optim as optim
import itertools

n_epochs = 200
decay_epoch=100
# hyperparams - Adam
lr= 0.0002
beta1= 0.5
beta2= 0.999

# Optimizers
g_optimizer = optim.Adam(itertools.chain(G_XtoY.parameters(), G_YtoX.parameters()), lr, [beta1, beta2])
d_x_optimizer = optim.Adam(D_X.parameters(), lr, [beta1, beta2])
d_y_optimizer = optim.Adam(D_Y.parameters(), lr, [beta1, beta2])

# LR schedulers
lr_scheduler_g = torch.optim.lr_scheduler.LambdaLR(g_optimizer, lr_lambda=LRLambda(n_epochs, decay_epoch).step)
lr_scheduler_d_x = torch.optim.lr_scheduler.LambdaLR(d_x_optimizer, lr_lambda=LRLambda(n_epochs, decay_epoch).step)
lr_scheduler_d_y = torch.optim.lr_scheduler.LambdaLR(d_y_optimizer, lr_lambda=LRLambda(n_epochs, decay_epoch).step)

In [0]:
# restore checkpoint 

# epoch_checkpoint = 79

# checkpoint_ = torch.load(f"/content/drive/My Drive/Colab Notebooks/RNA/Proyecto/CycleGan/Checkpoints/{model_name}_d_x_epoch_{epoch_checkpoint}")
# D_X.load_state_dict(checkpoint_['model_state_dict'])
# d_x_optimizer.load_state_dict(checkpoint_['optimizer_state_dict'])

# checkpoint_ = torch.load(f"/content/drive/My Drive/Colab Notebooks/RNA/Proyecto/CycleGan/Checkpoints/{model_name}_d_y_epoch_{epoch_checkpoint}")
# D_Y.load_state_dict(checkpoint_['model_state_dict'])
# d_y_optimizer.load_state_dict(checkpoint_['optimizer_state_dict'])

# checkpoint_ = torch.load(f"/content/drive/My Drive/Colab Notebooks/RNA/Proyecto/CycleGan/Checkpoints/{model_name}_g_x2y_epoch_{epoch_checkpoint}")
# G_XtoY.load_state_dict(checkpoint_['model_state_dict'])
# g_optimizer.load_state_dict(checkpoint_['optimizer_state_dict'])

# checkpoint_ = torch.load(f"/content/drive/My Drive/Colab Notebooks/RNA/Proyecto/CycleGan/Checkpoints/{model_name}_g_y2x_epoch_{epoch_checkpoint}")
# G_YtoX.load_state_dict(checkpoint_['model_state_dict'])

<All keys matched successfully>

In [0]:
def training(dataloader_X, dataloader_Y, test_X, test_Y, path_experiments, n_epochs=1000, n_eval_epochs=10):
  a_d_x_loss = np.zeros(n_epochs)
  a_d_y_loss = np.zeros(n_epochs)
  a_g_total_loss = np.zeros(n_epochs)

  a_gan_y2x_total_loss = np.zeros(n_epochs)
  a_gan_x2y_total_loss = np.zeros(n_epochs)
  a_cycle_y_total_loss = np.zeros(n_epochs)
  a_cycle_x_total_loss = np.zeros(n_epochs)
  a_identity_y_total_loss = np.zeros(n_epochs)
  a_identity_x_total_loss = np.zeros(n_epochs)

  test_X = test_X.to(device)
  test_Y = test_Y.to(device)
  for idx_epoch in range(0, n_epochs):
    aux_d_x_loss = 0
    aux_d_y_loss = 0
    aux_g_total_loss = 0
    #num_samples_x = 0
    #num_samples_y = 0
    for idx_batch, (images_X, images_Y) in enumerate(zip(dataloader_X, dataloader_Y)):
      #print(f"e:{idx_epoch} b:{idx_batch}")
      batch_size_x = images_X.size()[0]
      batch_size_y = images_Y.size()[0]
      images_X = images_X.to(device)
      images_Y = images_Y.to(device)
      #num_samples_x += batch_size_x
      #num_samples_y += batch_size_y

      # labels discriminator X
      target_real_x = torch.ones(batch_size_x, device=device, requires_grad=False)
      target_fake_x = torch.zeros(batch_size_x, device=device, requires_grad=False)
      # labels discriminator Y
      target_real_y = torch.ones(batch_size_y, device=device, requires_grad=False)
      target_fake_y = torch.zeros(batch_size_y, device=device, requires_grad=False)

      # Generator
      g_optimizer.zero_grad()
      # GAN loss: engañar discriminador
      fake_X = G_YtoX(images_Y) # fake X
      gan_loss_y2x = criterion_GAN(D_X(fake_X), target_real_y) # erorr fake x (como real X)
      fake_Y = G_XtoY(images_X) # fake y
      gan_loss_x2y = criterion_GAN(D_Y(fake_Y), target_real_x) # error fake y (como real y)
      # CYCLE loss: reconstruccion
      reconstructed_Y = G_XtoY(fake_X) # reconstructed Y
      cycle_loss_y = criterion_cycle(images_Y, reconstructed_Y) * 10 # erorr reconstructed y
      reconstructed_X = G_YtoX(fake_Y) # reconstructed x
      cycle_loss_x = criterion_cycle(images_X, reconstructed_X) * 10 # erorr reconstructed x
      # IDENTITY loss: devuelve misma imgen, ya que la entrada es lo que sale decodificador
      identity_loss_y = criterion_identity(images_Y, G_XtoY(images_Y)) * 5 
      identity_loss_x = criterion_identity(images_X, G_YtoX(images_X)) * 5

      a_gan_y2x_total_loss[idx_epoch] += gan_loss_y2x
      a_gan_x2y_total_loss[idx_epoch] += gan_loss_x2y
      a_cycle_y_total_loss[idx_epoch] += cycle_loss_y
      a_cycle_x_total_loss[idx_epoch] += cycle_loss_x
      a_identity_y_total_loss[idx_epoch] += identity_loss_y
      a_identity_x_total_loss[idx_epoch] += identity_loss_x

      # TOTAL loss
      g_total_loss = gan_loss_y2x + cycle_loss_y + gan_loss_x2y + cycle_loss_x + identity_loss_y + identity_loss_x
      aux_g_total_loss += g_total_loss
      g_total_loss.backward()
      g_optimizer.step()
      
      # Discriminator X
      d_x_optimizer.zero_grad()
      real_D_x_loss = criterion_GAN(D_X(images_X), target_real_x)
      fake_D_x_loss = criterion_GAN(D_X(G_YtoX(images_Y)), target_fake_y)
      d_x_loss = (real_D_x_loss + fake_D_x_loss) * 0.5
      aux_d_x_loss += d_x_loss
      d_x_loss.backward()
      d_x_optimizer.step()
      
      # Discriminator Y
      d_y_optimizer.zero_grad()
      real_D_y_loss = criterion_GAN(D_Y(images_Y), target_real_y)
      fake_D_y_loss = criterion_GAN(D_Y(G_XtoY(images_X)), target_fake_x)
      d_y_loss = (real_D_y_loss + fake_D_y_loss) * 0.5
      aux_d_y_loss += d_y_loss
      d_y_loss.backward()
      d_y_optimizer.step()

    a_d_x_loss[idx_epoch] = aux_d_x_loss /  (idx_batch + 1) #/ num_samples_x
    a_d_y_loss[idx_epoch] = aux_d_y_loss  / (idx_batch + 1) #/  num_samples_y
    a_g_total_loss[idx_epoch] = aux_g_total_loss / (idx_batch + 1) #/ (num_samples_x + num_samples_y)
    
    print(f'Epoch [{idx_epoch}/{n_epochs}] | d_X_loss: {a_d_x_loss[idx_epoch]:6.4f} | d_Y_loss: {a_d_y_loss[idx_epoch]:6.4f} | g_total_loss: {a_g_total_loss[idx_epoch]:6.4f}')

    if (idx_epoch+1) % n_eval_epochs == 0:
      generated_Y = (G_XtoY(test_X).to("cpu") + 1.0) * 0.5 # desnormalizar
      generated_X = (G_YtoX(test_Y).to("cpu") + 1.0) * 0.5 # desnormalizar
      path_reference_epoch = f"{path_experiments}/epoch_{idx_epoch}"
      if not os.path.isdir(path_reference_epoch):
        os.makedirs(path_reference_epoch)
      for idx_ in range(generated_Y.size()[0]):
        save_image(generated_Y[idx_, :, :, :],f"{path_reference_epoch}/Y_{idx_}.png") 
        save_image(generated_X[idx_, :, :, :],f"{path_reference_epoch}/X_{idx_}.png") 

      # save models
      if not os.path.isdir("/content/drive/My Drive/Colab Notebooks/RNA/Proyecto/CycleGanDA/Checkpoints/"):
        os.makedirs("/content/drive/My Drive/Colab Notebooks/RNA/Proyecto/CycleGanDA/Checkpoints/")
      torch.save({'epoch': idx_epoch, 'model_state_dict': D_X.state_dict(), 'optimizer_state_dict': d_x_optimizer.state_dict()},f"/content/drive/My Drive/Colab Notebooks/RNA/Proyecto/CycleGanDA/Checkpoints/{model_name}_d_x_epoch_{idx_epoch}")
      torch.save({'epoch': idx_epoch, 'model_state_dict': D_Y.state_dict(), 'optimizer_state_dict': d_y_optimizer.state_dict()},f"/content/drive/My Drive/Colab Notebooks/RNA/Proyecto/CycleGanDA/Checkpoints/{model_name}_d_y_epoch_{idx_epoch}")
      torch.save({'epoch': idx_epoch, 'model_state_dict': G_XtoY.state_dict(), 'optimizer_state_dict': g_optimizer.state_dict()},f"/content/drive/My Drive/Colab Notebooks/RNA/Proyecto/CycleGanDA/Checkpoints/{model_name}_g_x2y_epoch_{idx_epoch}")
      torch.save({'epoch': idx_epoch, 'model_state_dict': G_YtoX.state_dict(), 'optimizer_state_dict': g_optimizer.state_dict()},f"/content/drive/My Drive/Colab Notebooks/RNA/Proyecto/CycleGanDA/Checkpoints/{model_name}_g_y2x_epoch_{idx_epoch}")

    lr_scheduler_g.step()
    lr_scheduler_d_x.step()
    lr_scheduler_d_y.step()

  return a_d_x_loss, a_d_y_loss, a_g_total_loss, a_gan_y2x_total_loss, a_gan_x2y_total_loss, a_cycle_y_total_loss, a_cycle_x_total_loss, a_identity_y_total_loss, a_identity_x_total_loss


In [0]:
import os 
from torchvision.utils import save_image
# reference_images during training
val_apple_iter = iter(dataloader_test_apple)
val_apple_images = next(val_apple_iter)

val_orange_iter = iter(dataloader_test_orange)
val_orange_images = next(val_orange_iter)

path_reference_images = "/content/drive/My Drive/Colab Notebooks/RNA/Proyecto/CycleGanDA/Resultados/reference"

if not os.path.isdir(path_reference_images):
        os.makedirs(path_reference_images)

for idx_ in range(val_apple_images.size()[0]):
  save_image((val_apple_images[idx_, :, :, :]+1)*0.5,f"{path_reference_images}/apple_{idx_}.png") 
  save_image((val_orange_images[idx_, :, :, :]+1)*0.5,f"{path_reference_images}/orange_{idx_}.png") 

In [0]:
path_experimentos = "/content/drive/My Drive/Colab Notebooks/RNA/Proyecto/CycleGanDA/Resultados/experimentos"
d_x_loss_a, d_y_loss_a, g_loss_a, gan_y2x_loss_a, gan_x2y_loss_a, cycle_y_loss_a, cycle_x_loss_a, identity_y_loss_a, identity_x_loss_a = training(dataloader_train_apple, dataloader_train_orange, val_apple_images, val_orange_images, path_experimentos, n_epochs=n_epochs)

Epoch [0/200] | d_X_loss: 11.8464 | d_Y_loss: 9.9755 | g_total_loss: 601.1691
Epoch [1/200] | d_X_loss: 12.1532 | d_Y_loss: 8.5630 | g_total_loss: 526.7641
Epoch [2/200] | d_X_loss: 11.8676 | d_Y_loss: 9.1516 | g_total_loss: 496.8557
Epoch [3/200] | d_X_loss: 11.4433 | d_Y_loss: 9.0121 | g_total_loss: 471.1977
Epoch [4/200] | d_X_loss: 11.2255 | d_Y_loss: 8.5408 | g_total_loss: 459.4330
Epoch [5/200] | d_X_loss: 11.3095 | d_Y_loss: 6.5489 | g_total_loss: 449.9630
Epoch [6/200] | d_X_loss: 11.1312 | d_Y_loss: 6.9982 | g_total_loss: 451.4426
Epoch [7/200] | d_X_loss: 10.6798 | d_Y_loss: 6.2132 | g_total_loss: 436.2909
Epoch [8/200] | d_X_loss: 11.0513 | d_Y_loss: 5.4211 | g_total_loss: 435.7534
Epoch [9/200] | d_X_loss: 10.7602 | d_Y_loss: 5.7404 | g_total_loss: 417.8188
Epoch [10/200] | d_X_loss: 10.6366 | d_Y_loss: 4.8247 | g_total_loss: 419.8027
Epoch [11/200] | d_X_loss: 10.7174 | d_Y_loss: 5.1260 | g_total_loss: 405.7886
Epoch [12/200] | d_X_loss: 10.4810 | d_Y_loss: 6.2040 | g_tota

# **Plots**

In [0]:
import matplotlib.pyplot as plt

def linear_plot_train_validation(values, metric, filename, loc="lower right"):
  x_ = [i+1for i in range(values_train.shape[0])]
  plt.xlabel("epochs")
  plt.ylabel(metric)
  plt.plot(values)
  plt.legend(loc=loc)
  plt.savefig(filename)
  plt.close()

dir_plots=f"/content/drive/My Drive/Colab Notebooks/RNA/Proyecto/plots/{model_name}/"
if not os.path.exists(dir_plots):
    os.makedirs(dir_plots)

linear_plot_train_validation(d_x_loss_a, "Loss Discriminator X", f"{dir_plots}/d_x_loss.pdf")
linear_plot_train_validation(d_y_loss_a, "Loss Discriminator Y", f"{dir_plots}/d_y_loss.pdf")
linear_plot_train_validation(g_loss_a, "Loss Generator", f"{dir_plots}/g_total_loss.pdf")

linear_plot_train_validation(gan_y2x_loss_a, "Loss GAN Y2X", f"{dir_plots}/gan_y2x_loss.pdf")
linear_plot_train_validation(gan_x2y_loss_a, "Loss GAN X2Y", f"{dir_plots}/gan_x2y_loss.pdf")

linear_plot_train_validation(cycle_y_loss_a, "Cycle loss Y", f"{dir_plots}/cycle_y_loss.pdf")
linear_plot_train_validation(cycle_x_loss_a, "Cycle loss X", f"{dir_plots}/cycle_x_loss.pdf")

linear_plot_train_validation(identity_x_loss_a, "Identity loss X", f"{dir_plots}/identity_x_loss.pdf")
linear_plot_train_validation(identity_y_loss_a, "Identity loss Y", f"{dir_plots}/identity_y_loss.pdf")




# **Metric Values**

In [0]:
path_metrics_values = f"/content/drive/My Drive/Colab Notebooks/RNA/Proyecto/metrics_values/{model_name}/"

if not os.path.exists(path_metrics_values):
    os.makedirs(path_metrics_values)

np.savetxt(f'{path_metrics_values}loss_discriminator_x.txt', d_x_loss_a, delimiter=',', fmt="%.6f")
np.savetxt(f'{path_metrics_values}loss_discriminator_y.txt', d_y_loss_a, delimiter=',', fmt="%.6f")
np.savetxt(f'{path_metrics_values}loss_generator.txt', g_loss_a, delimiter=',', fmt="%.6f")

np.savetxt(f'{path_metrics_values}gan_y2x_loss.txt', gan_y2x_loss_a, delimiter=',', fmt="%.6f")
np.savetxt(f'{path_metrics_values}gan_x2y_loss.txt', gan_x2y_loss_a, delimiter=',', fmt="%.6f")

np.savetxt(f'{path_metrics_values}cycle_x_loss.txt', cycle_x_loss_a, delimiter=',', fmt="%.6f")
np.savetxt(f'{path_metrics_values}cycle_y_loss.txt', cycle_y_loss_a, delimiter=',', fmt="%.6f")

np.savetxt(f'{path_metrics_values}identity_x_loss.txt', identity_x_loss_a, delimiter=',', fmt="%.6f")
np.savetxt(f'{path_metrics_values}identity_y_loss.txt', identity_y_loss_a, delimiter=',', fmt="%.6f")